# Modify RTL based on NSGA II suggested solution points

In [5]:
import numpy as np
import re
from typing import Tuple


class PEArrayModifier:
    """
    A class to modify PE (Processing Element) arrays in Verilog files based on input array values.
    """
    def __init__(self, array_4096: np.ndarray, verilog_file: str = "PE_groups.v"):
        """
        Initialize the PE Array Modifier
        
        Args:
            array_4096: NumPy array of 4096 elements (values 0-9)
            verilog_file: Path to the Verilog file to modify
        """
        if len(array_4096) != 4096:
            raise ValueError("Array must have exactly 4096 elements")
        if not np.all((array_4096 >= 0) & (array_4096 <= 9)):
            raise ValueError("All array elements must be between 0 and 9 (inclusive)")
            
        self.array_4096 = array_4096
        self.verilog_file = verilog_file
        self.modifications_made = 0
        self.patterns_not_found = 0
        self.modification_log = []
    
    def index_to_coordinates(self, index: int) -> Tuple[int, int]:
        """
        Convert linear index to 64x64 grid coordinates
        
        Args:
            index: Linear index (0-4095)
            
        Returns:
            Tuple of (x, y) coordinates
        """
        x = index // 64
        y = index % 64
        return x, y


    # Convert 64x64 grid coordinates to linear index
    def coordinates_to_index(self, x: int, y: int) -> int:
        return x * 64 + y

    # Modify the Verilog file based on the input array
    def modify_verilog_file(self) -> bool:
        try:
            with open(self.verilog_file, 'r') as f:
                content = f.read()
            
            # Keep track of original content for backup
            original_content = content
            
            # Process each element in the array
            for index in range(4096):
                x, y = self.index_to_coordinates(index)
                new_value = self.array_4096[index]
                
                # Pattern to match the MODIFY_HERE comment
                modify_pattern = rf'// MODIFY_HERE_{x}__{y}'
                
                # Find all matches for this PE
                matches = list(re.finditer(modify_pattern, content))
                
                if matches:
                    for match in matches:
                        # Find the line after the MODIFY_HERE comment
                        match_end = match.end()
                        next_line_start = content.find('\n', match_end) + 1
                        if next_line_start == 0:  # No newline found
                            continue
                            
                        # Find the end of the next line
                        next_line_end = content.find('\n', next_line_start)
                        if next_line_end == -1:
                            next_line_end = len(content)
                        
                        next_line = content[next_line_start:next_line_end]
                        
                        # Pattern for approx module instantiation
                        approx_pattern = r'approx_\d+'
                        approx_match = re.search(approx_pattern, next_line)
                        
                        if approx_match:
                            # Replace the number after approx_
                            old_approx = approx_match.group()
                            new_approx = f'approx_{new_value}'
                            
                            # Replace in the content
                            new_line = next_line.replace(old_approx, new_approx, 1)
                            content = content[:next_line_start] + new_line + content[next_line_end:]
                            
                            # Log the modification
                            self.modification_log.append({
                                'index': index,
                                'coordinates': (x, y),
                                'old_value': old_approx,
                                'new_value': new_approx,
                                'line_content': new_line.strip(),
                                'modify_comment': match.group()
                            })
                            self.modifications_made += 1
                            
                            # Debug print for first few modifications
                            if self.modifications_made <= 10:
                                print(f"DEBUG: Modified index {index} ({x},{y}): {old_approx} -> {new_approx}")
                                print(f"       Modify comment: {match.group()}")
                                print(f"       New line: {new_line.strip()}")
                        else:
                            self.patterns_not_found += 1
                            print(f"Warning: No approx_ pattern found for PE at index {index} ({x}, {y})")
                            print(f"         Next line content: '{next_line.strip()}'")
                else:
                    self.patterns_not_found += 1
                    print(f"Warning: MODIFY_HERE pattern not found for index {index} ({x}, {y})")
                
                # Print progress every 100 PEs
                if (index + 1) % 100 == 0:
                    self.print_progress_report(index + 1)
            
            # Write the modified content back to file
            with open(self.verilog_file, 'w') as f:
                f.write(content)
            
            # Create backup of original
            backup_file = self.verilog_file + '.backup'
            with open(backup_file, 'w') as f:
                f.write(original_content)
            
            print(f"\nModification complete!")
            print(f"Total modifications made: {self.modifications_made}")
            print(f"Patterns not found: {self.patterns_not_found}")
            print(f"Original file backed up as: {backup_file}")
            
            return True
            
        except FileNotFoundError:
            print(f"Error: File '{self.verilog_file}' not found")
            return False
        except Exception as e:
            print(f"Error modifying file: {str(e)}")
            return False


    #  sample_indices: List of indices to test (default: first 10)
    def debug_pattern_search(self, sample_indices: list = None) -> None:
        if sample_indices is None:
            sample_indices = list(range(10))
        try:
            with open(self.verilog_file, 'r') as f:
                content = f.read()
            
            print("=== PATTERN MATCHING DEBUG ===")
            
            for index in sample_indices:
                x, y = self.index_to_coordinates(index)
                print(f"\nIndex {index} -> Coordinates ({x}, {y})")
                
                # Test the MODIFY_HERE pattern
                pattern = rf'// MODIFY_HERE_{x}__{y}'
                matches = list(re.finditer(pattern, content))
                print(f"  Pattern: {pattern}")
                print(f"    Matches found: {len(matches)}")
                
                if matches:
                    for match in matches[:2]:  # Show first 2 matches
                        print(f"    Match: '{match.group()}'")
                        
                        # Check the next line
                        match_end = match.end()
                        next_line_start = content.find('\n', match_end) + 1
                        if next_line_start > 0:
                            next_line_end = content.find('\n', next_line_start)
                            if next_line_end == -1:
                                next_line_end = len(content)
                            next_line = content[next_line_start:next_line_end]
                            print(f"    Next line: '{next_line.strip()}'")
                            
                            # Check for approx pattern
                            approx_match = re.search(r'approx_\d+', next_line)
                            if approx_match:
                                print(f"    Found approx: '{approx_match.group()}'")
                            else:
                                print(f"    No approx pattern found in next line")
                
                if index >= 4:  # Limit debug output
                    print("... (limiting debug output)")
                    break
                    
        except FileNotFoundError:
            print(f"Error: File '{self.verilog_file}' not found")
        except Exception as e:
            print(f"Error in debug: {str(e)}")
    

    # processed_count: Number of PEs processed so far
    def print_progress_report(self, processed_count: int):
        start_idx = processed_count - 100
        end_idx = processed_count - 1
        
        print(f"\n--- Progress Report: PEs {start_idx}-{end_idx} ---")
        print(f"Modifications in this batch: {len([m for m in self.modification_log if start_idx <= m['index'] <= end_idx])}")
        print(f"Patterns not found so far: {self.patterns_not_found}")
    
    def display_array_info(self):
        print("=== ARRAY INFORMATION ===")
        print(f"Array shape: {self.array_4096.shape}")
        print(f"Min value: {np.min(self.array_4096)}")
        print(f"Max value: {np.max(self.array_4096)}")
        print(f"Most common value: {np.bincount(self.array_4096).argmax()} (count: {np.bincount(self.array_4096).max()})")


# Generate a random NumPy array of 4096 elements with values between 0 and 9
def generate_random_array() -> np.ndarray:
    return np.random.randint(0, 10, size=4096, dtype=np.int8)


if __name__ == "__main__":
    test_array = np.loadtxt('array_config.dat', dtype=np.int8)
    test_array = generate_random_array()
    
    # Create modifier instance
    modifier = PEArrayModifier(test_array, "PE_groups.v")
    
    # Display array info
    modifier.display_array_info()
    
    # Run debug first
    print("\nRunning pattern matching debug...")
    modifier.debug_pattern_search([0, 1, 63, 64, 127, 4095])
    
    # Run actual modification
    print("\nRunning modification...")
    success = modifier.modify_verilog_file()
    
    if success:
        print("\nModification completed successfully!")
        print(f"Total modifications: {modifier.modifications_made}")
        print(f"Patterns not found: {modifier.patterns_not_found}")

=== ARRAY INFORMATION ===
Array shape: (4096,)
Min value: 0
Max value: 9
Most common value: 2 (count: 429)

Running pattern matching debug...
=== PATTERN MATCHING DEBUG ===

Index 0 -> Coordinates (0, 0)
  Pattern: // MODIFY_HERE_0__0
    Matches found: 1
    Match: '// MODIFY_HERE_0__0'
    Next line: 'approx_1 approx_LL_1(.x(input_to_a_approx), .y(input_to_b_approx), .Y(approx_out));'
    Found approx: 'approx_1'

Index 1 -> Coordinates (0, 1)
  Pattern: // MODIFY_HERE_0__1
    Matches found: 11
    Match: '// MODIFY_HERE_0__1'
    Next line: 'approx_1 approx_LL_2(.x(input_to_a_approx), .y(input_to_b_approx), .Y(approx_out));'
    Found approx: 'approx_1'
    Match: '// MODIFY_HERE_0__1'
    Next line: 'approx_1 approx_LL_2(.x(input_to_a_approx), .y(input_to_b_approx), .Y(approx_out));'
    Found approx: 'approx_1'

Index 63 -> Coordinates (0, 63)
  Pattern: // MODIFY_HERE_0__63
    Matches found: 1
    Match: '// MODIFY_HERE_0__63'
    Next line: 'approx_1 approx_LL_1(.x(input_to_a_

# Simpler code that is going to be broadcasted to all remotes

In [ ]:
import numpy as np
import re

class PEArrayModifier:
    def __init__(self, array_4096, verilog_file="./rtl/top.v"):
        self.array = array_4096
        self.file = verilog_file
    
    def modify(self):
        with open(self.file, 'r') as f:
            content = f.read()
        
        for idx in range(4096):
            x, y = idx//64, idx%64
            new_val = self.array[idx]
            pattern = rf'// MODIFY_HERE_{x}__{y}'
            
            for match in re.finditer(pattern, content):
                start = match.end()
                next_line_start = content.find('\n', start) + 1
                if next_line_start == 0: continue
                
                next_line_end = content.find('\n', next_line_start)
                if next_line_end == -1: next_line_end = len(content)
                
                line = content[next_line_start:next_line_end]
                old = re.search(r'approx_\d+', line)
                if old:
                    new_line = line.replace(old.group(), f'approx_{new_val}', 1)
                    print(f"PE_{x}_{y}: {old.group()} -> approx_{new_val}")
                    content = content[:next_line_start] + new_line + content[next_line_end:]
        
        with open(self.file, 'w') as f:
            f.write(content)

if __name__ == "__main__":
    arr = np.loadtxt('array_config.dat', dtype=np.int8)
    modifier = PEArrayModifier(arr)
    modifier.modify()

PE_0_0: approx_5 -> approx_5
PE_0_1: approx_5 -> approx_5
PE_0_1: approx_7 -> approx_5
PE_0_1: approx_5 -> approx_5
PE_0_1: approx_3 -> approx_5
PE_0_1: approx_1 -> approx_5
PE_0_1: approx_2 -> approx_5
PE_0_1: approx_9 -> approx_5
PE_0_1: approx_8 -> approx_5
PE_0_1: approx_9 -> approx_5
PE_0_1: approx_5 -> approx_5
PE_0_1: approx_6 -> approx_5
PE_0_2: approx_8 -> approx_8
PE_0_2: approx_5 -> approx_8
PE_0_2: approx_1 -> approx_8
PE_0_2: approx_7 -> approx_8
PE_0_2: approx_5 -> approx_8
PE_0_2: approx_5 -> approx_8
PE_0_2: approx_4 -> approx_8
PE_0_2: approx_6 -> approx_8
PE_0_2: approx_6 -> approx_8
PE_0_2: approx_6 -> approx_8
PE_0_2: approx_1 -> approx_8
PE_0_3: approx_9 -> approx_9
PE_0_3: approx_1 -> approx_9
PE_0_3: approx_4 -> approx_9
PE_0_3: approx_7 -> approx_9
PE_0_3: approx_7 -> approx_9
PE_0_3: approx_9 -> approx_9
PE_0_3: approx_2 -> approx_9
PE_0_3: approx_2 -> approx_9
PE_0_3: approx_7 -> approx_9
PE_0_3: approx_8 -> approx_9
PE_0_3: approx_9 -> approx_9
PE_0_4: approx

Traceback (most recent call last):
  File "/home/nira/miniconda3/envs/ml_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3546, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_216380/2207369394.py", line 39, in <module>
    modifier.modify()
  File "/tmp/ipykernel_216380/2207369394.py", line None, in modify
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nira/miniconda3/envs/ml_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nira/miniconda3/envs/ml_env/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nira/miniconda3/envs/ml_env/lib

# Generating a test array_config.dat

In [ ]:
import numpy as np
import os

output_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
output_filename = 'array_config.dat'
output_full_path = os.path.join(output_directory, output_filename)

os.makedirs(output_directory, exist_ok=True)

# Generate a NumPy array of 4096 random integers
# np.random.randint(low, high, size, dtype) generates integers from low (inclusive)
# to high (exclusive). So, 0 to 10 will give values 0, 1, ..., 9.
# dtype=np.int8 specifies the data type.
random_elements = np.random.randint(0, 10, size=4096, dtype=np.int8)

# Save the array to the file
# reshape(1, -1) ensures it's written as a single row of numbers (Nx1 matrix becomes 1xN)
# fmt='%d' ensures integers are saved without decimal points
# delimiter=' ' separates each number with a space
np.savetxt(output_full_path, random_elements.reshape(1, -1), fmt='%d', delimiter=' ')

print(f"Successfully generated '{output_full_path}' with {random_elements.size} random integers (0-9).")
print(f"First 10 elements of generated array: {random_elements[:10]}")

Successfully generated '/home/nira/Documents/code/ece/Strassen_MatMul_GEMM_NN_Accel/src/array_config.dat' with 4096 random integers (0-9).
First 10 elements of generated array: [8 2 8 5 2 8 4 7 6 3]
